In [ ]:
##########################################

# Start here once given anonymized csvs.

###########################################

import pandas as pd
import numpy as np
import os
import LLB_custom_scripts_mac
import csv

In [ ]:
# Loop through anonymized files and save the dataframes 
# of progression and module files in separate dictionaries.
# Also make a dictionary with the number of assessments in 
# each module, each year.

# Define paths for locating anonymized csvs.
anon_module_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                        "Data/anonymized module csvs/")

anon_progression_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                            "Data/anonymized progression csvs/")

progression_files = {}
module_files = {}
n_assessments_module_dict = {}

for filename in os.listdir(anon_module_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        module_name = f'{filename}'.split('.')[0]
        module_files[module_name] = pd.read_csv(
            anon_module_csv_path + f'{filename}')
        module_files[module_name].set_index('SPR Code', inplace=True)
        num_assessments = module_files[module_name].columns[-1].split(' ')[1]
        n_assessments_module_dict[module_name] = num_assessments
for filename in os.listdir(anon_progression_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        progression_files[f'{filename}'.split('.')[0]] = pd.read_csv(
            anon_progression_csv_path + f'{filename}')
        progression_files[f'{filename}'.split('.')[0]].set_index(
            'SPR Code', inplace=True)

In [ ]:
#################################################################

# Below are code snippets that I will turn into functions for batch
# analysis once I've gotten the snippets figured out and have 
# planned the batch analysis process. 
#
# Each snippet is for a single module.
#
# Each snippet should be written as efficiently as possible.

#################################################################

In [ ]:
# Get names of related module tables 
# initial assessment, reassessment, retake
module_name = '201617_CONTRACT_assessment'
if module_name.split('_')[0] == '201718':
    retake_name = None
else:
    retake_name = module_name.split('_')[0][:2] + \
        str(int(module_name.split('_')[0][2:4]) + 1) + \
        str(int(module_name.split('_')[0][4:]) + 1) + '_' + \
        module_name.split('_')[1] + '_' + module_name.split('_')[2]

# Determine if module 
for index, row in module_files[module_name].iterrows():
    if retake_name != None:
        if row['Result'] == 'F': 
            if index in module_files[retake_name].index.values:
#                 print(module_files[retake_name].at[index, 'Attempt'])
                if int(module_files[retake_name].at[index, 'Attempt']) == int(module_files[module_name].at[index, 'Attempt']) + 1:
                    module_files[module_name].at[index,'Retake'] = 'Y'
                elif int(module_files[retake_name].at[index, 'Attempt']) == int(module_files[module_name].at[index, 'Attempt']) + 2:
                    module_files[module_name].at[index,'Reassess'] = 'Y'
                else:
                    module_files[module_name].at[index,'Retake'] = 'error'
        
# values = []

# for value in module_files[module_name].index.values:
#     if value in module_files[retake_name].index.values:
#         values.append(value)
# values

module_files[module_name][['Result','Retake','Reassess']]

# what is happening to the students that are not reassessing or retaking??

In [ ]:
# determine all possibilities for part-time program label and full-time program label
program_names = set()
for dfname, df in module_files.items():
    program_names = set(df['Programme']) | program_names
program_names

In [ ]:
# Finds the number of instances of each type of student in the data
# Has one instance each time a student took a classs
# Shows that LLB full and part time are the biggest contributors (98% of instances)
# If we ignore the other students, that is fine

# Also, the total number of instances is 6972 (found by summing counts here)

'''
    Need to 'go through' df['Programme'] 
    and then count how many in each using pandas
    .value_counts which makes a series,
    and then join the series of each df
'''

X = pd.DataFrame(data = None, index = list(program_names))

for dfname, df in module_files.items():
    X[dfname] = df['Programme'].value_counts()
    
counts = X.sum(axis=1)
percents = (counts/counts.sum()) * 100
percents

In [ ]:
counts.sum()

In [ ]:
# Let me try to import the PT or FT tag from the progression tables again

for dfname, df in module_files.items():
    year, module, assessment = dfname.split('_') 
    progression_table_PT = progression_files[f'{year}_PROGRESSION_PT']
    progression_table_FT = progression_files[f'{year}_PROGRESSION_FT']
    progression_table = pd.concat([progression_table_PT, progression_table_FT])
    df = df.join(progression_table['Occur.'], how='left', rsuffix='2')
    module_files[dfname] = df


In [ ]:
module_files['201516_LAND_assessment'].sort_index()

In [ ]:
# some are missing
# need to assess which ones are missing - is there a pattern?
X = pd.DataFrame()
for dfname, df in module_files.items():
    X[dfname] = df['Occur.'].value_counts(normalize=True, dropna = False)

X

In [ ]:
# are all of the progression students accounted for in one of the sheets?

# make a set of SPR codes that are in the modules
module_SPR_codes = set()
    
for df in progression_files.values():
    module_SPR_codes = set(df.index.values) | module_SPR_codes
    
for dfname, df in progression_files.items():
    df['In_module'] = pd.Series(df.index.values) in module_SPR_codes


In [ ]:
# Determine which assessments were failed by students who failed
# a module and calculate the proportionate share each assessment
# contributed to students' module failures.

# Make a df to store assessment results
num_assessments = int(n_assessments_module_dict[example_module])
df_column_names = [f'Assessment {a} P or F' \
                   for a in range(1, num_assessments + 1)]
df_idx = module_files[example_module].index
P_F_status = pd.DataFrame(data=[], index=df_idx, \
                             columns = df_column_names)

# Populate df
for index, row in module_files[example_module].iterrows():
    for a in range(1,num_assessments+1):
        if row[f'Assessment {str(a)} Grade'] in ['P','LP']:
            P_F_status.at[index, f'Assessment {str(a)} P or F'] = 'P'
        elif row[f'Assessment {str(a)} Grade'] in ['F', 'W', 'FR', 'DR']:
            P_F_status.at[index, f'Assessment {str(a)} P or F'] = 'F'
        

new_df = P_F_status.join(module_files[example_module])

new_df

In [ ]:
# Determine if each student sat each failed assessment
# 
# The below is for an example module. 
# Make sub-size dataframe called submission_status
num_assessments = int(n_assessments_module_dict[example_module])
column_names = \
    [f'Assessment {a} Submission' for a in range(1, num_assessments + 1)]
idx = module_files[example_module].index
submission_status = pd.DataFrame(data=[], index=idx, columns = column_names)

for index, row in module_files[example_module].iterrows():
    for assessment in range(1,num_assessments+1):
        if row[f'Assessment {assessment} Grade'] != 'P':
            try:
                if int(float(row[f'Assessment {assessment} Mark'])) == 0:
                    submission_status.at[index, f'Assessment {assessment} Submission'] = \
                    'Non-submission'
                else:
                    submission_status.at[index, f'Assessment {assessment} Submission'] = \
                    'Failed submission'
            except:
                continue
submission_status.join(module_files[example_module])
# Then, can join onto existing df and make a count? Or make new df

In [ ]:
# how many failed assessment 1

failed_a1 = len(P_F_status.loc[P_F_status['Assessment 1 P or F'] \
                                                      == 'F'])
failed_a1

# # how many failed assessment 2

# failed_a2 = len(P_F_status.loc[P_F_status['Assessment 2 P or F'] \
#                                                         == 'F'])

# how many got no grade at all?


In [ ]:
##### Will need to deal with marks being float sometimes
int(float('33.0'))

In [ ]:
###### Make module summary file

In [ ]:
# Count number of students each module's assessment and store in module
# summary file.


In [ ]:
# For each assessment and reassessment table, add variable
# for whether progressed after reassessment or retake
# if applicable. Calculate percentage of students retaking
# and progressing at the end of retake. Ditto for reassess.
# Store in module summary.

In [ ]:
# For each assessment file, add a column for EACH ASSESSMENT 
# retake verus reassess (FR or DR) versus NA, \ 
# then count re-assess versus retake and percentage
# and store percentage in module summary file.



In [ ]:
# Look at re-assessment period for the same year, and 
# mark whether each student is DR or FR for each 
# assignment, based on previous assessment's data
# then use groupby to determine stats on passing module
# (sum function for P versus F?) 
# Store in module summary file

In [ ]:
# For each assessment and reassessment, 
# add a column for non-submission 
# versus failed submission versus NA, then count each
# overall, and for each individual assessment/reassessment
# store in module summary file